### Case description

Imagine you as a project manager; one thing you might do on a daily basis is to look at a status report of the project. Unfortunately, that report is in Excel file, and to make thing worse, there are hundreds of line items that aren't to your interest. Sure, Excel has a filter function, but that will be too much repetitive work if you do the filtering on a daily basis. Also, new items can be added in today's report but not present in the past report. How are you going to catch this combined with other hundreds of line items after filtering in Excel?\
\
Below, we will demonstrate how Python and Pandas together can help us summarize status changes of the items we are interested in and output a Excel summary. Let's start!

### Import libraries

In [261]:
import pandas as pd
from pandas import ExcelWriter
pd.set_option('display.max_columns', None) #force pandas to display all columns
pd.set_option('display.max_rows', None)    #force pandas to display all rows, CAREFUL
import numpy as np
from datetime import datetime

### Import Excel files

First, we import 2 Excel files, one has all the line items from yesterday, or last week, and the other one has all the line items from today. They have the same column setups but may have different content.

In [262]:
oldTable = pd.read_excel('report yesterday.xlsx', sheet_name='Sheet1')  # report from yesterday
newTable = pd.read_excel('report today.xlsx', sheet_name='Sheet1')  # report from today

print('Excel table dimension:\n', 'old table:', oldTable.shape, '\n', 'new table:', newTable.shape)

colList = list(oldTable.columns)
print('\n','The Excel file has '+ str(len(colList)) + ' columns:\n', colList)

Excel table dimension:
 old table: (718, 20) 
 new table: (585, 20)

 The Excel file has 20 columns:
 ['IssueID', 'Issue Status', 'Severity', 'Target Closure Date', 'Green Checks', 'Blue Checks', 'Red Checks', 'Total', 'Mfg Functional Area', 'Lead PMT', 'Date Opened', 'Date Closed', 'Date Last Modified', 'Issue Additional Information', 'Additional Information', 'Originating CoP', 'Causal Factors', 'Resolution Lead', 'UnUp', 'ICNI_Title']


### Preprocessing
The imported raw Excel file above has 21 columns, and about 700 line items. We select the columns we are interested in and filter by a specific team we want to look at; also, perform data type conversion for target columns.

In [263]:
colToKeep = ['IssueID', 'Issue Status', 'Target Closure Date', 'Green Checks', 'Blue Checks', 'Red Checks', 'Total', 'Mfg Functional Area', 'Lead PMT', 'Issue Description', 'Action', 'Date Opened', 'Date Last Modified',  'Resolution Lead']
oldTable = oldTable.filter(items=colToKeep)
newTable = newTable.filter(items=colToKeep)

oldTable = oldTable[(oldTable['Lead PMT'] == '4 - Powertrain')]
newTable = newTable[(newTable['Lead PMT'] == '4 - Powertrain')]

oldTable['IssueID'] = oldTable['IssueID'].astype(int)
newTable['IssueID'] = newTable['IssueID'].astype(int)

oldTable['Date Opened'] = pd.to_datetime(oldTable['Date Opened'], infer_datetime_format=True, format = '%Y-%m-%d',errors='raise')
newTable['Date Opened'] = pd.to_datetime(newTable['Date Opened'], infer_datetime_format=True, format = '%Y-%m-%d',errors='raise')


### Compare the old and new tables to see if the issue status column is changed since the last report, if yes, indicator = 1
Here comes the difference between manual laborious manipulation in Excel and the swift operation by Pandas.

In [264]:
oldStatus = oldTable[['IssueID', 'Issue Status']]
newStatus = newTable[['IssueID', 'Issue Status']]

statusChange = pd.merge(oldStatus, newStatus, on = 'IssueID', how = 'outer')
display(statusChange.head())

def findChange(x):
    if x['Issue Status_x'] != x['Issue Status_y']:
        return 1
    else:
        return 0

statusChange['status change indicator'] = statusChange.apply(findChange, axis=1)
statusChange = statusChange.rename(columns={'Issue Status_x': 'Issue Status OLD', 'Issue Status_y': 'Issue Status NEW'})
display(statusChange.head())

,IssueID,Issue Status_x,Issue Status_y
0,194376,closed,closed
1,202594,closed,closed
2,194385,closed,closed
3,202899,closed,closed
4,194384,closed,closed


,IssueID,Issue Status OLD,Issue Status NEW,status change indicator
0,194376,closed,closed,0
1,202594,closed,closed,0
2,194385,closed,closed,0
3,202899,closed,closed,0
4,194384,closed,closed,0


### Merge the status change results into the new table

In [265]:
output = pd.merge(newTable, statusChange, on = 'IssueID', how = 'outer')
output.fillna('NA', inplace=True)  # fill NA if there are new issueID generated in today report or issueID gets deleted
output.drop(['Issue Status'], axis=1,inplace=True)
display(output.head())

,IssueID,Target Closure Date,Green Checks,Blue Checks,Red Checks,Total,Mfg Functional Area,Lead PMT,Date Opened,Date Last Modified,Resolution Lead,Issue Status OLD,Issue Status NEW,status change indicator
0,194376,08-May-2020,45,0,0,45,Final Assembly,4 - Powertrain,2019-12-26,06-May-2020,CAD,closed,closed,0
1,202594,10-Apr-2020,45,0,0,45,Final Assembly,4 - Powertrain,2020-03-11,08-Apr-2020,CAD,closed,closed,0
2,194385,28-Feb-2020,44,0,0,44,Final Assembly,4 - Powertrain,2019-12-26,19-Feb-2020,CAD,closed,closed,0
3,202899,24-Apr-2020,42,0,0,42,Final Assembly,4 - Powertrain,2020-03-13,21-Apr-2020,CAD,closed,closed,0
4,194384,27-Feb-2020,41,0,0,41,Final Assembly,4 - Powertrain,2019-12-26,19-Feb-2020,CAD,closed,closed,0


### Reorder table by issueID

In [266]:
output = output.sort_values(by=['IssueID','Issue Status NEW'], ascending = True)

### One more filtering
We want to focus on any open or pending items or the items status have changed between the old and new report.

In [267]:
output = output[(output['Issue Status NEW'] != 'closed') | (output['status change indicator'] == 1)]

### Make a issue summary table
A quick summary table shows how many items are in open or pending status.

In [268]:
summary = output.groupby(['Issue Status NEW'], as_index=False)['Issue Status OLD'].count()
summary.rename(columns={"Issue Status OLD": "Count"}, inplace=True)
display(summary)

,Issue Status NEW,Count
0,open,9
1,pending,2


### Color coding Issue Status column
highlight the status column using different color coding for easier differentiation.

In [269]:
def color(val):
    if val == 'closed':
        color = '#5fba7d'
    elif val == 'pending':
        color = 'yellow'
    elif val == 'open':
        color = '#d65f5f'
    elif val == 'NA':
        color = '#d65f5f'
    return 'background-color: %s' % color

output = output.style.applymap(color, subset=['Issue Status OLD','Issue Status NEW'])

### Export a cleaned status report

In [270]:
today = datetime.today().strftime('%Y-%m-%d')
fileName = 'report ' + today + '.xlsx'

writer = ExcelWriter(fileName, date_format = 'yyyy-mm-dd',datetime_format='yyyy-mm-dd')
output.to_excel(writer,'Sheet1', index=False)
summary.to_excel(writer,'Summary', index=False)
writer.save()

### Conclusion

We successfully generated a cleaned and brief summary report cater to our needs.\
This may not seem to be a big deal at first glance; however, imagine you as a project manager who wants to track the status on a weekly or daily basis, then, using this tool can help save hundreds of miniutes.